In [5]:
import numpy as np
from scipy.stats import norm

def run_sdt_var(n_trials=200, d_prime=1.5, std_nontarget=1, std_target=1.5**0.5, criterion_c=0, epsilon=0.0001):
    mean_nontarget = 0
    mean_target = mean_nontarget + d_prime * std_nontarget  # 기준은 여전히 non-target 기준

    criterion = mean_nontarget + criterion_c * std_nontarget

    responses = []
    labels = []

    for _ in range(n_trials):
        if np.random.rand() < 0.5:
            sensory_input = np.random.normal(mean_target, std_target)
            labels.append(1)
        else:
            sensory_input = np.random.normal(mean_nontarget, std_nontarget)
            labels.append(0)

        if sensory_input >= criterion:
            responses.append(1)
        else:
            responses.append(0)

    # Hit / False Alarm
    hits = 0
    false_alarms = 0

    for i in range(len(responses)):
        r = responses[i]
        l = labels[i]

        if r == 1 and l == 1:
            hits += 1

        if r == 1 and l == 0:
            false_alarms += 1

    n_targets = sum(labels)
    n_nontargets = len(labels) - n_targets

    # rate 계산
    if n_targets > 0:
        hit_rate = hits / n_targets
    else:
        hit_rate = epsilon

    if n_nontargets > 0:
        false_alarm_rate = false_alarms / n_nontargets
    else:
        false_alarm_rate = epsilon

    # 확률 보정
    hit_rate = min(max(hit_rate, epsilon), 1 - epsilon)
    false_alarm_rate = min(max(false_alarm_rate, epsilon), 1 - epsilon)

    # d', c 계산
    z_hit = norm.ppf(hit_rate)
    z_fa = norm.ppf(false_alarm_rate)
    d_empirical = z_hit - z_fa
    c_empirical = -0.5 * (z_hit + z_fa)

    print("[criterion c =", criterion_c, "]")
    print("추정된 d':", round(d_empirical, 2))
    print("추정된 c:", round(c_empirical, 2), "\n")

    return d_empirical, c_empirical

In [9]:
run_sdt_var(criterion_c=0);
run_sdt_var(criterion_c=1);
run_sdt_var(criterion_c=-1);


[criterion c = 0 ]
추정된 d': 1.45
추정된 c: -0.76 

[criterion c = 1 ]
추정된 d': 1.29
추정된 c: 0.25 

[criterion c = -1 ]
추정된 d': 2.61
추정된 c: -2.42 

